In [117]:
import polars as pl

In [119]:
df_preppet = pl.read_parquet('kjoretoyinfo_preppet.parquet')

In [126]:
df_preppet

tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_aksler_drift,tekn_merke,tekn_modell,tekn_drivstoff,tekn_neste_pkk,er_elektrisk,tekn_farge,tekn_kjtgrp
date,date,i64,i64,str,str,date,bool,str,i64
2021-10-13,2021-10-13,2,3900,"""VITO""","""2""",2025-10-13,false,"""05""",315
2020-01-29,2020-01-29,2,5750,"""CADDY""","""2""",2026-01-29,false,"""06""",315
2023-11-29,2023-11-29,1,2600,"""Mustang Mach-E…","""5""",2027-11-29,true,"""06""",101
2020-01-27,2020-01-27,2,5750,"""CADDY""","""2""",2024-01-27,false,"""01""",315
2019-01-30,2019-01-30,2,2600,"""F-150 Platinum…","""2""",2024-05-03,false,"""06""",315
…,…,…,…,…,…,…,…,…,…
2003-11-27,2017-06-10,0,4350,"""ZAFIRA-A""","""1""",2021-04-30,false,"""11""",101
2014-01-17,2018-07-31,2,2600,"""RANGER""","""2""",2024-06-30,false,"""05""",315
2008-03-31,2023-07-28,0,3930,"""MX-5""","""1""",2025-08-16,false,"""05""",101


In [127]:
   df = df_preppet.with_columns([
        pl.col("tekn_reg_f_g_n").cast(pl.Utf8).str.to_date(format="%Y%m%d").alias("tekn_reg_f_g_n"),
        pl.col("tekn_reg_eier_dato").cast(pl.Utf8).str.to_date(format="%Y%m%d").alias("tekn_reg_eier_dato"),
        pl.col("tekn_neste_pkk").cast(pl.Utf8).str.to_date(format="%Y%m%d", strict=False).alias("tekn_neste_pkk")
    ])

ComputeError: conversion from `str` to `date` failed in column 'tekn_reg_f_g_n' for 2692915 out of 2692915 values: ["2021-10-13", "2020-01-29", … "2004-09-21"]

You might want to try:
- setting `strict=False` to set values that cannot be converted to `null`
- using `str.strptime`, `str.to_date`, or `str.to_datetime` and providing a format string

In [ ]:
date_2022_01_01 = date(2022, 1, 1)

In [ ]:
df_2022_01_01 = df_preppet.filter(pl.col('tekn_reg_eier_dato') == date_2022_01_01)

In [ ]:
print(df_2022_01_01)

In [ ]:
df_2022_01_01 = df_preppet.filter(pl.col('tekn_reg_eier_dato') == '2022-01-01')

In [113]:
print(df_2022_01_01)

NameError: name 'df_2022_01_01' is not defined

In [97]:
df_traktorer = df_preppet.filter(df_preppet['tekn_kjtgrp'] == 401)

In [106]:
df_2022 = df_traktorer.filter(pl.col("tekn_reg_eier_dato").dt.year() == 2022)

In [107]:
mest_populær_farge=df_2022.group_by("tekn_farge").agg(
    pl.col("tekn_farge").count().alias("antall_forekomster")
).sort("antall_forekomster", descending=True).head(1)


In [108]:
print(mest_populær_farge)

shape: (1, 2)
┌────────────┬────────────────────┐
│ tekn_farge ┆ antall_forekomster │
│ ---        ┆ ---                │
│ str        ┆ u32                │
╞════════════╪════════════════════╡
│ 04         ┆ 3238               │
└────────────┴────────────────────┘
